In [3]:
from neo4j import GraphDatabase
import pandas as pd

host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'letmein'
driver = GraphDatabase.driver(host,auth=(user, password))

def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [ ]:
neo4j_query("""
CREATE CONSTRAINT IF NOT EXISTS ON (n: `NFT`) ASSERT n.`ID_token` IS UNIQUE;
""")

In [ ]:
neo4j_query("""
USING PERIODIC COMMIT 50000
LOAD CSV WITH HEADERS FROM "file:///Data_API.csv" AS record
WITH apoc.map.clean(record,[],[""," ","-"]) AS record
// UNWIND $records AS record
MERGE (n: `NFT` { `ID_token`: record.`Unique_id_collection` })
SET n.`Smart_contract` = record.`Smart_contract`
SET n.`Image_url_1` = record.`Image_url_1`
SET n.`Image_url_2` = record.`Image_url_2`
SET n.`Image_url_3` = record.`Image_url_3`
SET n.`Image_url_4` = record.`Image_url_4`
SET n.`Name` = record.`Name`
SET n.`Description` = record.`Description`
//SET n.`Market` = record.`Market`
SET n.`Permanent_link` = record.`Permanent_link`;
""")

In [2]:
neo4j_query("""
CREATE CONSTRAINT IF NOT EXISTS ON (n: `Trader`) ASSERT n.`address` IS UNIQUE;
""")

""


In [3]:
neo4j_query("""
USING PERIODIC COMMIT 50000
LOAD CSV WITH HEADERS FROM "file:///Data_API.csv" AS record
WITH apoc.map.clean(record,[],[""," ","-"]) AS record
// UNWIND $records AS record
MERGE (n: `Trader` { `address`: record.`Buyer_address` })
SET n.`username` = record.`Buyer_username`;
""")

""


In [6]:
neo4j_query("""
USING PERIODIC COMMIT 50000
LOAD CSV WITH HEADERS FROM "file:///Data_API.csv" AS record
WITH apoc.map.clean(record,[],[""," ","-"]) AS record
// UNWIND $records AS record
MERGE (n: `Trader` { `address`: record.`Seller_address` })
SET n.`username` = record.`Seller_username`;
""")

""


In [7]:
neo4j_query("""
CREATE INDEX ON :Transaction(Transaction_hash);
""")

""


In [9]:
neo4j_query("""
USING PERIODIC COMMIT 50000
LOAD CSV WITH HEADERS FROM "file:///Data_API.csv" AS record
WITH apoc.map.clean(record,[],[""," ","-"]) AS record
MATCH (seller: `Trader` { `address`: record.`Seller_address` })
MATCH (buyer: `Trader` { `address`: record.`Buyer_address` })
MATCH (nft: `NFT` { `ID_token`: record.`Unique_id_collection` })

CREATE (n: `Transaction` { `Transaction_hash`: record.`Transaction_hash` })
SET n.`Price_Crypto` = toFloat(record.`Price_Crypto`)
SET n.`Crypto` = record.`Crypto`
SET n.`Price_USD` = toFloat(record.`Price_USD`)
SET n.`Datetime_updated` = datetime(replace(record.`Datetime_updated`,' ','T'))
SET n.`Datetime_updated_seconds` = datetime(replace(record.`Datetime_updated_seconds`,' ', 'T'))
SET n.`Market` = record.`Market`
MERGE (seller)-[:SOLD]->(n)
MERGE (buyer)-[:BOUGHT]->(n)
MERGE (n)-[:FOR_NFT]->(nft)
;
""")

""


In [10]:
neo4j_query("""
CREATE CONSTRAINT IF NOT EXISTS ON (n: `Collection`) ASSERT n.`Collection` IS UNIQUE;
""")

""


In [11]:
neo4j_query("""
USING PERIODIC COMMIT 50000
LOAD CSV WITH HEADERS FROM "file:///Data_API.csv" AS record
WITH apoc.map.clean(record,[],[""," ","-"]) AS record
MERGE (n: `Collection` { `Collection`: record.`Collection_cleaned` })
;
""")

""


In [12]:
neo4j_query("""
CREATE CONSTRAINT IF NOT EXISTS ON (n: `Category`) ASSERT n.`Category` IS UNIQUE;
""")

""


In [13]:
neo4j_query("""
USING PERIODIC COMMIT 50000
LOAD CSV WITH HEADERS FROM "file:///Data_API.csv" AS record
WITH apoc.map.clean(record,[],[""," ","-"]) AS record
MERGE (n: `Category` { `Category`: record.`Category` });
""")

""


In [4]:
neo4j_query("""
USING PERIODIC COMMIT 50000
LOAD CSV WITH HEADERS FROM "file:///Data_API.csv" AS record
WITH apoc.map.clean(record,[],[""," ","-"]) AS record
MATCH (source: `NFT` { `ID_token`: record.`Unique_id_collection` })
MATCH (target: `Collection` { `Collection`: record.`Collection_cleaned` })
MERGE (source)-[r: `IN_COLLECTION`]->(target);
""")

""


In [5]:
neo4j_query("""
USING PERIODIC COMMIT 50000
LOAD CSV WITH HEADERS FROM "file:///Data_API.csv" AS record
WITH apoc.map.clean(record,[],[""," ","-"]) AS record
MATCH (source: `NFT` { `ID_token`: record.`ID_token` })
MATCH (target: `Category` { `Category`: record.`Category` })
MERGE (source)-[r: `IN_CATEGORY`]->(target);
""")

""


In [6]:
neo4j_query("""
CREATE INDEX ON :Trader(username);
""")

neo4j_query("""
CREATE INDEX ON :Transaction(Price_USD);
""")

neo4j_query("""
CREATE INDEX ON :Transaction(Datetime_updated_seconds);
""")

neo4j_query("""
CREATE INDEX ON :NFT(Name);
""")

neo4j_query("""
CREATE INDEX ON :NFT(Description);
""")

""
